In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers

In [2]:
train_datagen =ImageDataGenerator(
                                rescale=1./255,
                                rotation_range=30,
                                shear_range=0.3,
                                zoom_range=0.3,
                                width_shift_range=0.4,
                                height_shift_range=0.4,
                                horizontal_flip=True,
                                fill_mode="nearest")

In [3]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
                                                    './train/',
                                                    color_mode="grayscale",
                                                    batch_size=64,
                                                    class_mode="categorical",
                                                    shuffle=True)

Found 24177 images belonging to 5 classes.


In [5]:
validation_generator = validation_datagen.flow_from_directory(  
                                                    './validation/',
                                                    color_mode="grayscale",
                                                    batch_size=64,
                                                    class_mode="categorical",
                                                    shuffle=True)

Found 3006 images belonging to 5 classes.


In [6]:
visible = Input(shape=(256,256,1))
conv1 = Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1, 1))(visible)
conv2 = Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1, 1))(conv1)
bat1 = BatchNormalization()(conv2)
zero1 = ZeroPadding2D(padding=(1, 1))(bat1)

conv3 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(zero1)
conv4 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(conv3)
bat2 = BatchNormalization()(conv4)

conv5 = Conv2D(64, kernel_size=(3,3), activation='relu',strides=(1, 1), padding='valid')(bat2)
conv6 = Conv2D(64, kernel_size=(3,3), activation='relu',strides=(1, 1), padding='valid')(conv5)
bat3 = BatchNormalization()(conv6)
pool1 = MaxPooling2D(pool_size=(2, 2))(bat3)
zero2 = ZeroPadding2D(padding=(1, 1))(pool1)

conv7 = Conv2D(128, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.01))(zero2)
conv8 = Conv2D(128, kernel_size=(2,2), activation='relu', strides=(1, 1), padding='valid')(conv7)
bat4 = BatchNormalization()(conv8)

conv15 = Conv2D(128, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(bat4)
conv16 = Conv2D(128, kernel_size=(2,2), activation='relu', strides=(1, 1), padding='valid')(conv8)
bat8 = BatchNormalization()(conv16)

flat = Flatten()(bat8)
hidden1 = Dense(32, activation='relu')(flat)
drop1 = Dropout(0.3)(hidden1)

hidden2 = Dense(32, activation='relu')(drop1)
drop2 = Dropout(0.2)(hidden2)

output = Dense(5, activation='sigmoid')(drop2)

In [7]:
model = Model(inputs=visible, outputs=output)

In [8]:
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [9]:
model.compile(optimizer= opt, loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 16)      2320      
_________________________________________________________________
batch_normalization (BatchNo (None, 252, 252, 16)      64        
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 254, 254, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 250, 250, 32)      9248  

In [11]:
checkpoint = ModelCheckpoint("Emotion_Model.h5",
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=4,
                         verbose=1,
                         restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

In [12]:
callback = [earlystop, checkpoint, reduce_lr]

In [ ]:
history = model.fit_generator(train_generator, epochs=25, steps_per_epoch=24177//64, callbacks=callback, validation_data=validation_generator, validation_steps=3006//64)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
 48/377 [==>...........................] - ETA: 2:11:32 - loss: 11.0549 - accuracy: 0.2236